In [5]:
import pandas as pd
from tabulate import tabulate
from fetch_process_function import fetch_and_save_erc20_transfers, process_transactions
'''
data_process.ipynb和data_fetch.ipynb同样功能，但是通过封装函数实现了给日期地址等参数就可以返回数据，更加简洁
添加了pnl的计算，但是对于某些交易判断还是有问题，导致结果计算不完全正确
'''

'\ndata_process.ipynb和data_fetch.ipynb同样功能，但是通过封装函数实现了给日期地址等参数就可以返回数据，更加简洁\n添加了pnl的计算，但是对于某些交易判断还是有问题，导致结果计算不完全正确\n'

In [6]:
def calculate_pnl(btc_record_df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate PnL for BTC transactions based on BTC_record_df.

    :param btc_record_df: DataFrame containing BTC transaction records.
    :return: DataFrame with calculated PnL and summary of Realised and Unrealised PnL.
    """
    
    # Define current price as the price of the last row
    current_price = btc_record_df.iloc[-1]['average_price']
    
    results = []
    total_pnl = 0

    for _, row in btc_record_df.iterrows():
        transaction_type = row['transaction_type']
        amount = row['other_token_volume']
        price = row['average_price']

        # Calculate PnL for each row based on the difference from the current price
        pnl = (current_price - price) * amount
        
        # For SELL transactions, PnL is negative because it's reducing the position
        if transaction_type == "SELL":
            pnl *= -1

        # Ignore the last row (current price reference)
        if row.name == btc_record_df.index[-1]:
            pnl = 0

        total_pnl += pnl

        # Append the result for this transaction
        results.append({
            "Date": row['dateTime'],
            "timeStamp": row['timeStamp'],
            "Type": transaction_type,
            "amount": amount,
            "price": price,
            "pnl": pnl
        })

    # Convert results to a DataFrame
    pnl_df = pd.DataFrame(results)

    return pnl_df


In [ ]:
ADDRESS = "0x5be9a4959308A0D0c7bC0870E319314d8D957dBB"
START_DATE = "2024-12-06 08:00" # Singapore Time，the function will convert it to UTC
END_DATE = "2024-12-12 08:00"
OUTPUT_FILE = "C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result"
API_KEY = "VQAIR728IM4Z8RZKPYBR4ESM5I3WBZK2C1"
BASE_TOKENS = {"USDT", "USDC", "USDE"}
transfers_df = fetch_and_save_erc20_transfers(
        address=ADDRESS,
        start_date=START_DATE,
        end_date=END_DATE,
        output_file=OUTPUT_FILE,
        offset=1000,
    )
transfers_df = pd.read_csv(OUTPUT_FILE + "/all_erc20_transfers.csv")
record_df_1, record_df_2, record_df_3 = process_transactions(transfers_df, OUTPUT_FILE, ADDRESS, BASE_TOKENS)
# print(tabulate(summary, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))
unique_tokens_count = record_df_2["token_symbol"].nunique()

unique_tokens = record_df_2["token_symbol"].unique()
results = []

for token in unique_tokens:
    token_df = record_df_2[record_df_2["formatted_record"].str.contains(fr"\b{token}\b", regex=True)]
    print(f"{token}: {len(token_df)} records")

    pnl_df = calculate_pnl(token_df)
    print(tabulate(pnl_df, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))   
    total_pnl = pnl_df["pnl"].sum()

    results.append({"token": token, "total_pnl": total_pnl})

pnl_summary_df = pd.DataFrame(results)

print(tabulate(pnl_summary_df, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))

# BTC_record_df = record_df_1[record_df_1["formatted_record"].str.contains(r"\bcbBTC\b", regex=True)]
# # print(BTC_record_df['formatted_record'])
# pnl_df = calculate_pnl(BTC_record_df)
# print(f"Total PnL: {pnl_df['pnl'].sum()}")




Formatted Transactions: part 1
Formatted Transactions: part 2
Formatted Transactions: final part
WLFI: 2 records
+---------------------+-------------+--------+---------------+---------+---------+
| Date                |   timeStamp | Type   |        amount |   price |     pnl |
+=====================+=============+========+===============+=========+=========+
| 2024-10-11 00:57:35 |  1728608255 | 'BUY'  | 2249999999.00 |    0.00 | 1795.30 |
+---------------------+-------------+--------+---------------+---------+---------+
| 2024-11-25 16:13:11 |  1732551191 | 'SELL' | 1000000000.00 |    0.00 |    0.00 |
+---------------------+-------------+--------+---------------+---------+---------+
cbBTC: 38 records
+---------------------+-------------+--------+----------+----------+--------+
| Date                |   timeStamp | Type   |   amount |    price |    pnl |
+=====================+=============+========+==========+==========+========+
| 2024-11-29 17:38:35 |  1732901915 | BUY    |     1.7

In [8]:
transfers_df = pd.read_csv(OUTPUT_FILE + "/all_erc20_transfers.csv")
# 分别统计 from 和 to 地址
from_address_counts = transfers_df["from"].value_counts().reset_index()
from_address_counts.columns = ["address", "from_count"]

to_address_counts = transfers_df["to"].value_counts().reset_index()
to_address_counts.columns = ["address", "to_count"]

address_counts = pd.merge(
    from_address_counts, to_address_counts, on="address", how="outer"
).fillna(0)
# merge and convert to int

address_counts["from_count"] = address_counts["from_count"].astype(int)
address_counts["to_count"] = address_counts["to_count"].astype(int)

print(address_counts)
address_counts.to_csv("from_to_address_counts.csv", index=False)


                                       address  from_count  to_count
0   0x0478aa0f766bbcfd0882ad8ae5df5f0531f06065           2         0
1   0x119ca458c5f91d3815e5f8a29633d709569b4eb3           0         1
2   0x1246e490308db61dca45ead613f47430de3830ad           0         2
3   0x2887f96c5b3dc08dbe1312017310b2f26b228e31           0         2
4   0x2ac62e416ce9fc52394a045b66dee9dead4be053           1         0
5   0x2d732631d248006f94b52af8fd0a610c85894d6a           1         0
6   0x33c59f63771f2ab4c9f20c3955526416f02528ca           0         2
7   0x3b54f4fbfafd0e7167dcd70594e2d1b8bcbfe4e9           0         2
8   0x4125af68487bec121a2b90c0e15d824b53899533           1         0
9   0x4ac3bf54fbbf481c991d2f04bad2ff6393d9eb42           1         0
10  0x5851a4148233d84fcbf255109f99bbdd6c8c9cba           0         2
11  0x5be9a4959308a0d0c7bc0870e319314d8d957dbb         112      4151
12  0x6ae4eb64fd04e36a006969135f5013cbb0c15285           1         0
13  0x6e3382c4c4a69a3a62b489a456b3